# Stock Price Prediction with Neural Networks

## Educational Notebook Accompanying Neural Networks Slides

This notebook demonstrates the concepts from the neural networks presentation by building a real stock price prediction model.

**Learning Path:**
1. Data Preparation
2. Neural Network from Scratch (Educational)
3. Progressive Complexity (1 → 2 → 3 layers)
4. Professional Implementation (PyTorch)
5. Results & Analysis

**Concepts Covered:**
- Forward Propagation (Slides 6-7)
- Loss Calculation (Slide 7)
- Backpropagation & Gradient Descent (Slide 8)
- Activation Functions (Slides 3-4)
- Overfitting/Underfitting (Slide 17)
- Learning Rate Effects (Slide 18)

## Part 1: Data Preparation

In [ ]:
# Install required packages (run once)
# !pip install yfinance pandas numpy matplotlib torch scikit-learn

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(42)

print("Libraries loaded successfully!")

In [ ]:
# Download Apple stock data
ticker = 'AAPL'
print(f"Downloading {ticker} stock data...")

data = yf.download(ticker, start='2019-01-01', end='2024-12-01', progress=False)
print(f"Downloaded {len(data)} days of data")

# Use closing prices
prices = data['Close'].values

# Visualize
plt.figure(figsize=(14, 5))
plt.plot(data.index, prices, linewidth=1)
plt.title(f'{ticker} Stock Price (2019-2024)', fontsize=14, fontweight='bold')
plt.xlabel('Date')
plt.ylabel('Price ($)')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"Price range: ${prices.min():.2f} - ${prices.max():.2f}")

In [ ]:
# Create features: Use past N days to predict next day
def create_sequences(data, lookback=5):
    """
    Create input-output sequences for time series prediction.
    
    Input (X): Past 'lookback' days of prices
    Output (y): Next day's price
    """
    X, y = [], []
    for i in range(len(data) - lookback):
        X.append(data[i:i+lookback])
        y.append(data[i+lookback])
    return np.array(X), np.array(y)

lookback = 10  # Use past 10 days to predict next day
X, y = create_sequences(prices, lookback)

print(f"Created {len(X)} training examples")
print(f"Input shape: {X.shape} (samples, features)")
print(f"Output shape: {y.shape}")
print(f"\nExample: Given prices {X[0]}, predict {y[0]:.2f}")

In [ ]:
# Split data: 70% train, 15% validation, 15% test
train_size = int(0.7 * len(X))
val_size = int(0.15 * len(X))

X_train, y_train = X[:train_size], y[:train_size]
X_val, y_val = X[train_size:train_size+val_size], y[train_size:train_size+val_size]
X_test, y_test = X[train_size+val_size:], y[train_size+val_size:]

print(f"Train set: {len(X_train)} samples")
print(f"Validation set: {len(X_val)} samples")
print(f"Test set: {len(X_test)} samples")

In [ ]:
# Normalize data to [0, 1] range (important for neural networks!)
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

X_train_scaled = scaler_X.fit_transform(X_train)
X_val_scaled = scaler_X.transform(X_val)
X_test_scaled = scaler_X.transform(X_test)

y_train_scaled = scaler_y.fit_transform(y_train.reshape(-1, 1)).flatten()
y_val_scaled = scaler_y.transform(y_val.reshape(-1, 1)).flatten()
y_test_scaled = scaler_y.transform(y_test.reshape(-1, 1)).flatten()

print("Data normalized to [0, 1] range")
print(f"Sample input (normalized): {X_train_scaled[0]}")
print(f"Sample output (normalized): {y_train_scaled[0]:.4f}")

## Part 2: Neural Network from Scratch

### Section 2A: Simple 1-Layer Network

**Architecture:**
- Input layer: 10 neurons (past 10 days)
- Hidden layer: 8 neurons (sigmoid activation)
- Output layer: 1 neuron (predicted price)

**Concepts demonstrated:**
- Forward propagation (Slide 6)
- Gradient descent (Slide 8)

In [ ]:
class SimpleNN:
    """Simple 1-layer neural network from scratch."""
    
    def __init__(self, input_size, hidden_size, output_size=1):
        # Initialize weights randomly (small values)
        self.W1 = np.random.randn(input_size, hidden_size) * 0.01
        self.b1 = np.zeros((1, hidden_size))
        self.W2 = np.random.randn(hidden_size, output_size) * 0.01
        self.b2 = np.zeros((1, output_size))
        
    def sigmoid(self, z):
        """Sigmoid activation function (Slide 3)."""
        return 1 / (1 + np.exp(-np.clip(z, -500, 500)))
    
    def sigmoid_derivative(self, z):
        """Derivative for backpropagation."""
        s = self.sigmoid(z)
        return s * (1 - s)
    
    def forward(self, X):
        """Forward propagation (Slide 6)."""
        # Hidden layer
        self.z1 = X @ self.W1 + self.b1
        self.a1 = self.sigmoid(self.z1)
        
        # Output layer (linear activation for regression)
        self.z2 = self.a1 @ self.W2 + self.b2
        self.a2 = self.z2  # Linear output
        
        return self.a2
    
    def compute_loss(self, y_true, y_pred):
        """Mean Squared Error loss (Slide 7)."""
        return np.mean((y_true - y_pred) ** 2)
    
    def backward(self, X, y_true, learning_rate=0.01):
        """Backpropagation (Slide 8)."""
        m = X.shape[0]
        
        # Output layer gradients
        dz2 = (self.a2.flatten() - y_true.flatten()).reshape(-1, 1)
        dW2 = (self.a1.T @ dz2) / m
        db2 = np.sum(dz2, axis=0, keepdims=True) / m
        
        # Hidden layer gradients
        dz1 = (dz2 @ self.W2.T) * self.sigmoid_derivative(self.z1)
        dW1 = (X.T @ dz1) / m
        db1 = np.sum(dz1, axis=0, keepdims=True) / m
        
        # Update weights (gradient descent)
        self.W1 -= learning_rate * dW1
        self.b1 -= learning_rate * db1
        self.W2 -= learning_rate * dW2
        self.b2 -= learning_rate * db2
    
    def train(self, X_train, y_train, X_val, y_val, epochs=1000, learning_rate=0.01, verbose=True):
        """Train the network."""
        train_losses = []
        val_losses = []
        
        for epoch in range(epochs):
            # Forward pass
            y_pred = self.forward(X_train)
            train_loss = self.compute_loss(y_train, y_pred.flatten())
            
            # Backward pass
            self.backward(X_train, y_train, learning_rate)
            
            # Validation loss
            y_val_pred = self.forward(X_val)
            val_loss = self.compute_loss(y_val, y_val_pred.flatten())
            
            train_losses.append(train_loss)
            val_losses.append(val_loss)
            
            if verbose and (epoch % 100 == 0 or epoch == epochs-1):
                print(f"Epoch {epoch:4d} | Train Loss: {train_loss:.6f} | Val Loss: {val_loss:.6f}")
        
        return train_losses, val_losses
    
    def predict(self, X):
        """Make predictions."""
        return self.forward(X).flatten()

print("SimpleNN class defined successfully!")

In [ ]:
# Train 1-layer network
print("Training 1-Layer Neural Network\n" + "="*50)

nn1 = SimpleNN(input_size=10, hidden_size=8)
train_losses_1, val_losses_1 = nn1.train(
    X_train_scaled, y_train_scaled,
    X_val_scaled, y_val_scaled,
    epochs=1000,
    learning_rate=0.1
)

# Plot training progress
plt.figure(figsize=(10, 4))
plt.plot(train_losses_1, label='Train Loss', linewidth=2)
plt.plot(val_losses_1, label='Validation Loss', linewidth=2)
plt.xlabel('Epoch')
plt.ylabel('Loss (MSE)')
plt.title('1-Layer Network: Training Progress', fontweight='bold')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

### Section 2B: 2-Layer Network (More Complexity)

**Architecture:**
- Input: 10 → Hidden1: 16 → Hidden2: 8 → Output: 1

In [ ]:
class DeepNN:
    """2-layer neural network."""
    
    def __init__(self, input_size, hidden1_size, hidden2_size, output_size=1):
        self.W1 = np.random.randn(input_size, hidden1_size) * 0.01
        self.b1 = np.zeros((1, hidden1_size))
        self.W2 = np.random.randn(hidden1_size, hidden2_size) * 0.01
        self.b2 = np.zeros((1, hidden2_size))
        self.W3 = np.random.randn(hidden2_size, output_size) * 0.01
        self.b3 = np.zeros((1, output_size))
    
    def sigmoid(self, z):
        return 1 / (1 + np.exp(-np.clip(z, -500, 500)))
    
    def sigmoid_derivative(self, z):
        s = self.sigmoid(z)
        return s * (1 - s)
    
    def forward(self, X):
        self.z1 = X @ self.W1 + self.b1
        self.a1 = self.sigmoid(self.z1)
        
        self.z2 = self.a1 @ self.W2 + self.b2
        self.a2 = self.sigmoid(self.z2)
        
        self.z3 = self.a2 @ self.W3 + self.b3
        self.a3 = self.z3
        
        return self.a3
    
    def compute_loss(self, y_true, y_pred):
        return np.mean((y_true - y_pred) ** 2)
    
    def backward(self, X, y_true, learning_rate=0.01):
        m = X.shape[0]
        
        dz3 = (self.a3.flatten() - y_true.flatten()).reshape(-1, 1)
        dW3 = (self.a2.T @ dz3) / m
        db3 = np.sum(dz3, axis=0, keepdims=True) / m
        
        dz2 = (dz3 @ self.W3.T) * self.sigmoid_derivative(self.z2)
        dW2 = (self.a1.T @ dz2) / m
        db2 = np.sum(dz2, axis=0, keepdims=True) / m
        
        dz1 = (dz2 @ self.W2.T) * self.sigmoid_derivative(self.z1)
        dW1 = (X.T @ dz1) / m
        db1 = np.sum(dz1, axis=0, keepdims=True) / m
        
        self.W1 -= learning_rate * dW1
        self.b1 -= learning_rate * db1
        self.W2 -= learning_rate * dW2
        self.b2 -= learning_rate * db2
        self.W3 -= learning_rate * dW3
        self.b3 -= learning_rate * db3
    
    def train(self, X_train, y_train, X_val, y_val, epochs=1000, learning_rate=0.01, verbose=True):
        train_losses = []
        val_losses = []
        
        for epoch in range(epochs):
            y_pred = self.forward(X_train)
            train_loss = self.compute_loss(y_train, y_pred.flatten())
            self.backward(X_train, y_train, learning_rate)
            
            y_val_pred = self.forward(X_val)
            val_loss = self.compute_loss(y_val, y_val_pred.flatten())
            
            train_losses.append(train_loss)
            val_losses.append(val_loss)
            
            if verbose and (epoch % 100 == 0 or epoch == epochs-1):
                print(f"Epoch {epoch:4d} | Train Loss: {train_loss:.6f} | Val Loss: {val_loss:.6f}")
        
        return train_losses, val_losses
    
    def predict(self, X):
        return self.forward(X).flatten()

print("DeepNN class defined!")

In [ ]:
# Train 2-layer network
print("Training 2-Layer Neural Network\n" + "="*50)

nn2 = DeepNN(input_size=10, hidden1_size=16, hidden2_size=8)
train_losses_2, val_losses_2 = nn2.train(
    X_train_scaled, y_train_scaled,
    X_val_scaled, y_val_scaled,
    epochs=1000,
    learning_rate=0.1
)

# Compare 1-layer vs 2-layer
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 4))

ax1.plot(train_losses_1, label='1-Layer', linewidth=2)
ax1.plot(train_losses_2, label='2-Layer', linewidth=2)
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Train Loss')
ax1.set_title('Training Loss Comparison', fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3)

ax2.plot(val_losses_1, label='1-Layer', linewidth=2)
ax2.plot(val_losses_2, label='2-Layer', linewidth=2)
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Validation Loss')
ax2.set_title('Validation Loss Comparison', fontweight='bold')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nFinal Train Loss - 1-Layer: {train_losses_1[-1]:.6f}, 2-Layer: {train_losses_2[-1]:.6f}")
print(f"Final Val Loss   - 1-Layer: {val_losses_1[-1]:.6f}, 2-Layer: {val_losses_2[-1]:.6f}")

### Section 2C: Evaluate on Test Set

Convert normalized predictions back to actual prices and calculate metrics.

In [ ]:
# Make predictions on test set
y_pred_1 = nn1.predict(X_test_scaled)
y_pred_2 = nn2.predict(X_test_scaled)

# Denormalize predictions
y_pred_1_actual = scaler_y.inverse_transform(y_pred_1.reshape(-1, 1)).flatten()
y_pred_2_actual = scaler_y.inverse_transform(y_pred_2.reshape(-1, 1)).flatten()

# Calculate metrics
mae_1 = mean_absolute_error(y_test, y_pred_1_actual)
rmse_1 = np.sqrt(mean_squared_error(y_test, y_pred_1_actual))

mae_2 = mean_absolute_error(y_test, y_pred_2_actual)
rmse_2 = np.sqrt(mean_squared_error(y_test, y_pred_2_actual))

print("Test Set Performance (From-Scratch Models)")
print("="*50)
print(f"1-Layer Network:")
print(f"  MAE:  ${mae_1:.2f}")
print(f"  RMSE: ${rmse_1:.2f}")
print(f"\n2-Layer Network:")
print(f"  MAE:  ${mae_2:.2f}")
print(f"  RMSE: ${rmse_2:.2f}")

# Visualize predictions
plt.figure(figsize=(14, 5))
plt.plot(y_test, label='Actual Price', linewidth=2, alpha=0.7)
plt.plot(y_pred_1_actual, label='1-Layer Prediction', linewidth=1.5, linestyle='--')
plt.plot(y_pred_2_actual, label='2-Layer Prediction', linewidth=1.5, linestyle='--')
plt.xlabel('Test Sample')
plt.ylabel('Price ($)')
plt.title('Test Set: Actual vs Predicted Prices', fontweight='bold')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## Part 3: Professional Implementation with PyTorch

Now let's implement the same models using PyTorch - a professional deep learning framework.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

# Convert numpy arrays to PyTorch tensors
X_train_torch = torch.FloatTensor(X_train_scaled)
y_train_torch = torch.FloatTensor(y_train_scaled).unsqueeze(1)
X_val_torch = torch.FloatTensor(X_val_scaled)
y_val_torch = torch.FloatTensor(y_val_scaled).unsqueeze(1)
X_test_torch = torch.FloatTensor(X_test_scaled)
y_test_torch = torch.FloatTensor(y_test_scaled).unsqueeze(1)

print("Data converted to PyTorch tensors")
print(f"X_train shape: {X_train_torch.shape}")
print(f"y_train shape: {y_train_torch.shape}")

In [ ]:
# Define PyTorch models
class PyTorchNN1(nn.Module):
    """1-Layer network in PyTorch."""
    def __init__(self, input_size=10, hidden_size=8):
        super(PyTorchNN1, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, 1)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        x = self.sigmoid(self.fc1(x))
        x = self.fc2(x)
        return x

class PyTorchNN2(nn.Module):
    """2-Layer network in PyTorch."""
    def __init__(self, input_size=10, hidden1_size=16, hidden2_size=8):
        super(PyTorchNN2, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden1_size)
        self.fc2 = nn.Linear(hidden1_size, hidden2_size)
        self.fc3 = nn.Linear(hidden2_size, 1)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        x = self.sigmoid(self.fc1(x))
        x = self.sigmoid(self.fc2(x))
        x = self.fc3(x)
        return x

class PyTorchNN3(nn.Module):
    """3-Layer network with ReLU (matching Slide 4)."""
    def __init__(self, input_size=10):
        super(PyTorchNN3, self).__init__()
        self.fc1 = nn.Linear(input_size, 32)
        self.fc2 = nn.Linear(32, 16)
        self.fc3 = nn.Linear(16, 8)
        self.fc4 = nn.Linear(8, 1)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.2)
    
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.relu(self.fc3(x))
        x = self.fc4(x)
        return x

print("PyTorch model classes defined!")

In [ ]:
def train_pytorch_model(model, X_train, y_train, X_val, y_val, epochs=1000, lr=0.01):
    """Train a PyTorch model."""
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    
    train_losses = []
    val_losses = []
    
    for epoch in range(epochs):
        # Training
        model.train()
        optimizer.zero_grad()
        y_pred = model(X_train)
        train_loss = criterion(y_pred, y_train)
        train_loss.backward()
        optimizer.step()
        
        # Validation
        model.eval()
        with torch.no_grad():
            y_val_pred = model(X_val)
            val_loss = criterion(y_val_pred, y_val)
        
        train_losses.append(train_loss.item())
        val_losses.append(val_loss.item())
        
        if epoch % 100 == 0 or epoch == epochs-1:
            print(f"Epoch {epoch:4d} | Train Loss: {train_loss.item():.6f} | Val Loss: {val_loss.item():.6f}")
    
    return train_losses, val_losses

print("Training function defined!")

In [ ]:
# Train all three PyTorch models
print("Training PyTorch 1-Layer Network\n" + "="*50)
pt_model_1 = PyTorchNN1()
pt_train_1, pt_val_1 = train_pytorch_model(pt_model_1, X_train_torch, y_train_torch, 
                                            X_val_torch, y_val_torch, epochs=1000, lr=0.01)

print("\nTraining PyTorch 2-Layer Network\n" + "="*50)
pt_model_2 = PyTorchNN2()
pt_train_2, pt_val_2 = train_pytorch_model(pt_model_2, X_train_torch, y_train_torch,
                                            X_val_torch, y_val_torch, epochs=1000, lr=0.01)

print("\nTraining PyTorch 3-Layer Network (ReLU)\n" + "="*50)
pt_model_3 = PyTorchNN3()
pt_train_3, pt_val_3 = train_pytorch_model(pt_model_3, X_train_torch, y_train_torch,
                                            X_val_torch, y_val_torch, epochs=1000, lr=0.001)

In [ ]:
# Compare all PyTorch models
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 4))

ax1.plot(pt_train_1, label='1-Layer', linewidth=2)
ax1.plot(pt_train_2, label='2-Layer', linewidth=2)
ax1.plot(pt_train_3, label='3-Layer (ReLU)', linewidth=2)
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Train Loss')
ax1.set_title('PyTorch Models: Training Loss', fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3)

ax2.plot(pt_val_1, label='1-Layer', linewidth=2)
ax2.plot(pt_val_2, label='2-Layer', linewidth=2)
ax2.plot(pt_val_3, label='3-Layer (ReLU)', linewidth=2)
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Validation Loss')
ax2.set_title('PyTorch Models: Validation Loss', fontweight='bold')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Evaluate PyTorch models on test set
pt_model_1.eval()
pt_model_2.eval()
pt_model_3.eval()

with torch.no_grad():
    y_pred_pt1 = pt_model_1(X_test_torch).numpy().flatten()
    y_pred_pt2 = pt_model_2(X_test_torch).numpy().flatten()
    y_pred_pt3 = pt_model_3(X_test_torch).numpy().flatten()

# Denormalize
y_pred_pt1_actual = scaler_y.inverse_transform(y_pred_pt1.reshape(-1, 1)).flatten()
y_pred_pt2_actual = scaler_y.inverse_transform(y_pred_pt2.reshape(-1, 1)).flatten()
y_pred_pt3_actual = scaler_y.inverse_transform(y_pred_pt3.reshape(-1, 1)).flatten()

# Calculate metrics
mae_pt1 = mean_absolute_error(y_test, y_pred_pt1_actual)
mae_pt2 = mean_absolute_error(y_test, y_pred_pt2_actual)
mae_pt3 = mean_absolute_error(y_test, y_pred_pt3_actual)

rmse_pt1 = np.sqrt(mean_squared_error(y_test, y_pred_pt1_actual))
rmse_pt2 = np.sqrt(mean_squared_error(y_test, y_pred_pt2_actual))
rmse_pt3 = np.sqrt(mean_squared_error(y_test, y_pred_pt3_actual))

print("PyTorch Models - Test Set Performance")
print("="*50)
print(f"1-Layer:  MAE=${mae_pt1:.2f}, RMSE=${rmse_pt1:.2f}")
print(f"2-Layer:  MAE=${mae_pt2:.2f}, RMSE=${rmse_pt2:.2f}")
print(f"3-Layer:  MAE=${mae_pt3:.2f}, RMSE=${rmse_pt3:.2f}")

## Part 4: Final Comparison & Analysis

Compare all models (from scratch vs PyTorch)

In [ ]:
# Summary table
results = pd.DataFrame({
    'Model': ['From Scratch - 1 Layer', 'From Scratch - 2 Layers', 
              'PyTorch - 1 Layer', 'PyTorch - 2 Layers', 'PyTorch - 3 Layers (ReLU)'],
    'MAE ($)': [mae_1, mae_2, mae_pt1, mae_pt2, mae_pt3],
    'RMSE ($)': [rmse_1, rmse_2, rmse_pt1, rmse_pt2, rmse_pt3]
})

print("\n" + "="*70)
print("FINAL COMPARISON: All Models")
print("="*70)
print(results.to_string(index=False))
print("="*70)

# Find best model
best_idx = results['MAE ($)'].idxmin()
print(f"\nBest Model: {results.loc[best_idx, 'Model']}")
print(f"  MAE: ${results.loc[best_idx, 'MAE ($)']:.2f}")
print(f"  RMSE: ${results.loc[best_idx, 'RMSE ($)']:.2f}")

In [ ]:
# Visualize all predictions
plt.figure(figsize=(14, 6))
plt.plot(y_test, label='Actual Price', linewidth=2.5, alpha=0.8, color='black')
plt.plot(y_pred_1_actual, label='Scratch 1-Layer', linewidth=1, linestyle='--', alpha=0.7)
plt.plot(y_pred_2_actual, label='Scratch 2-Layer', linewidth=1, linestyle='--', alpha=0.7)
plt.plot(y_pred_pt1_actual, label='PyTorch 1-Layer', linewidth=1, linestyle=':', alpha=0.7)
plt.plot(y_pred_pt2_actual, label='PyTorch 2-Layer', linewidth=1, linestyle=':', alpha=0.7)
plt.plot(y_pred_pt3_actual, label='PyTorch 3-Layer', linewidth=1.5, alpha=0.9)
plt.xlabel('Test Sample', fontsize=12)
plt.ylabel('AAPL Stock Price ($)', fontsize=12)
plt.title('All Models: Actual vs Predicted Prices', fontsize=14, fontweight='bold')
plt.legend(loc='best')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## Conclusions & Key Learnings

### Observations:

1. **Progressive Complexity**: Adding more layers generally improved performance, but with diminishing returns
2. **From Scratch vs Libraries**: PyTorch implementations converge faster and often achieve better results
3. **Activation Functions**: ReLU (Slide 4) outperformed sigmoid for deeper networks
4. **Learning Rate**: Lower learning rates (0.001 vs 0.1) worked better for deeper networks (Slide 18)

### Limitations of Stock Price Prediction:

- Stock prices are influenced by many factors beyond historical prices
- Markets are partially random (efficient market hypothesis)
- Past performance doesn't guarantee future results
- Neural networks can overfit to historical patterns

### Practical Takeaways:

1. **Start Simple**: Begin with simpler models before adding complexity
2. **Monitor Validation Loss**: Watch for overfitting (train loss ↓ but val loss ↑)
3. **Use Regularization**: Dropout helps prevent overfitting
4. **Choose Right Tools**: Use PyTorch/TensorFlow for production, build from scratch for learning

### Next Steps:

- Try different features (volume, moving averages, technical indicators)
- Experiment with LSTM networks (better for time series)
- Add more data (multiple stocks, longer history)
- Implement proper backtesting and risk management